In [1]:
#imports
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#gpu config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cpu


In [12]:
#hyperparams
learning_rate = 0.01
batch_size = 128
num_epochs = 30

#load data
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.FashionMNIST(root='data/', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='data/',train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [28]:
# Define the model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [29]:
#initialize
model = MLP().to(device)
writer = SummaryWriter()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [30]:
#Training

#train net
for epoch in range(num_epochs):
    train_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        
        # Send data to GPU
        images, labels = images.to(device), labels.to(device)

        # Forward pass and compute loss
        outputs = model(images)
        loss = nn.CrossEntropyLoss()(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Compute running train loss
        train_loss += loss.item()
        
    #validation
    model.eval()
    valid_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item()*data.size(0)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    
    valid_loss = valid_loss/len(test_loader.dataset)
    accuracy = 100 * correct / total

    writer.add_scalar('Training Loss', train_loss / len(train_loader), epoch)
    writer.add_scalar('Validation Loss', valid_loss / len(test_loader), epoch)
    
    print(f"Epoch {epoch+1}/{30}, Training Loss: {train_loss:.6f}, Validation Loss: {valid_loss:.6f}, Accuracy: {accuracy:.2f}")
writer.close()

Epoch 1/30, Training Loss: 820.545613, Validation Loss: 1.157388, Accuracy: 63.69
Epoch 2/30, Training Loss: 438.822264, Validation Loss: 0.818860, Accuracy: 67.94
Epoch 3/30, Training Loss: 350.947052, Validation Loss: 0.724914, Accuracy: 74.09
Epoch 4/30, Training Loss: 312.301404, Validation Loss: 0.657219, Accuracy: 77.19
Epoch 5/30, Training Loss: 284.985271, Validation Loss: 0.603409, Accuracy: 79.03
Epoch 6/30, Training Loss: 265.291680, Validation Loss: 0.573077, Accuracy: 79.96
Epoch 7/30, Training Loss: 251.333530, Validation Loss: 0.550649, Accuracy: 80.97
Epoch 8/30, Training Loss: 241.440320, Validation Loss: 0.531521, Accuracy: 81.46
Epoch 9/30, Training Loss: 233.768011, Validation Loss: 0.527236, Accuracy: 80.71
Epoch 10/30, Training Loss: 227.468612, Validation Loss: 0.504275, Accuracy: 82.20
Epoch 11/30, Training Loss: 223.012663, Validation Loss: 0.496714, Accuracy: 82.38
Epoch 12/30, Training Loss: 217.942980, Validation Loss: 0.488923, Accuracy: 82.53
Epoch 13/30, 